In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.client import device_lib
import os

In [ ]:
raw = pd.read_csv('face_dataset.csv')
raw_data = raw.values
raw_feature = raw_data[:, 0:10]
scaler = MinMaxScaler()
scaler.fit(raw_feature)

scaler.data_max_
print(scaler.data_max_,scaler.data_min_)
raw_feature = scaler.transform(raw_feature)
print(raw_feature)

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def print_history(history):
    # 绘制训练 & 验证的准确率值
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model accuracy&loss')
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.legend(['Train_acc', 'Val_acc', 'Train_loss', 'Val_loss'])
    plt.show()

def BP(lr):

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(40, activation='relu', input_shape=(9,)))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    model.build(input_shape=(None, 9))
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr),
                  metrics=['accuracy'])

    return model

In [ ]:
import keras
model_img_name = 'test.png'
keras.utils.plot_model(model, model_img_name, show_shapes=True,dpi=200)

In [ ]:
if __name__ == '__main__':

    data = []
    # 读取数据
    raw = pd.read_csv('face_dataset.csv')
    raw_data = raw.values
    raw_feature = raw_data[:, 0:10]

    # 数据归一化
    scaler = MinMaxScaler()
    scaler.fit(raw_feature)
    scaler.data_max_
    raw_feature = scaler.transform(raw_feature)

    # 将最后一列的缺陷类别转成one-hot编码形式
    x = []
    y = []
    for i in range(len(raw_feature)):
        x.append(list(raw_feature[i]))
        if raw_data[i][10] == 0:
            y.append([1, 0, 0])
        elif raw_data[i][10] == 1:
            y.append([0, 1, 0])
        elif raw_data[i][10] == 2:
            y.append([0, 0, 1])
            # 随机打乱数据
    x = np.array(x)
    y = np.array(y)
    permutation = np.random.permutation(len(x))
    x = x[permutation]
    y = y[permutation]

    # 选取打乱后的80%数据作为训练数据和验证数据
    train_data = x[0:2400]
    train_label = y[0:2400]

    test_data = x[2400:]
    test_label = y[2400:]

    lr = 0.005  #学习率初值，可动态下降
    bp_model = BP(lr=lr)

    bp_model.summary()

    #学习率动态衰减
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                      factor=0.8, patience=5,
                                                      min_lr=0.5e-7)

    # 早停法，保存训练中的最优参数
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                          verbose=0, patience=30, min_delta=0.0001,
                                          restore_best_weights='True')

    history = bp_model.fit(train_data, train_label, batch_size=10, epochs=1000, verbose=1,
                           callbacks=[lr_reducer, es], validation_split=0.2, shuffle=True)

In [ ]:

print_history(history)
print('loss, acc:', bp_model.evaluate(test_data, test_label, batch_size=100, verbose=0))

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score
test_true = np.argmax(test_label, axis=1)
test_pred = np.argmax(bp_model.predict(test_data), axis=1)
print("BP_Model Accuracy on test set: {:.4f}".format(accuracy_score (test_true, test_pred)))

In [ ]:
import pickle
with open('history.txt', 'wb', ) as file_txt:
    pickle.dump(history.history, file_txt)
# 模型保存.h5
bp_model.save('face_engage_76.h5')